In [12]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import reverse_geocoder as rg
import psycopg2 
from sqlalchemy import create_engine
import json

import warnings
warnings.filterwarnings('ignore')

In [2]:
def create_geoframe(dataset):
    lat = dataset['lat'][:]
    lon = dataset['lon'][:]
    lons, lats = np.meshgrid(lon, lat)
    lons_flatt = lons.flatten()
    lats_flatt = lats.flatten()
    whole_world = {
    'lon': lons_flatt, 
    'lat': lats_flatt, }
    geoframe = pd.DataFrame(whole_world)
    return geoframe

In [3]:
def create_column(dataset,name_variable):
    df_temp = pd.DataFrame()
    variable = dataset[name_variable][:]
    variable = variable[0]
    variable_flatt = variable.flatten()
    df_temp[name_variable] = variable_flatt
    return df_temp

In [4]:
def add_country(dataframe):
    coords = tuple(zip(dataframe['lat'], dataframe['lon']))
    results_rg = rg.search(coords)
    results_admin2 = [x.get('cc') for x in results_rg]
    dataframe['country'] = results_admin2
    return dataframe

In [5]:
def create_vegetation_class_breakdown(dataset):
    burned_area_in_vegetation_class= dataset['burned_area_in_vegetation_class'][:][0]
    i_vegetation_class = 10 
    df_temp = pd.DataFrame()
    for burned_area_per_veg_class in burned_area_in_vegetation_class:
        s_vegetation_class= f"{i_vegetation_class}_burned_area"
        burned_area_per_veg_class_flatt= burned_area_per_veg_class.flatten()
        df_temp[s_vegetation_class] = burned_area_per_veg_class_flatt
        i_vegetation_class= i_vegetation_class + 10
    df_temp['cropland_burned_area'] = df_temp['20_burned_area'] + df_temp['30_burned_area'] + df_temp['40_burned_area']
    df_temp['mosaic_tree_grass_burned_area'] = df_temp['100_burned_area'] + df_temp['100_burned_area']
    df_temp['other_vegetation_burned_area']= df_temp['120_burned_area'] + df_temp['130_burned_area'] + df_temp['140_burned_area']+ df_temp['150_burned_area']
    df_temp['flooded_area_burned_area']= df_temp['160_burned_area'] + df_temp['170_burned_area'] + df_temp['180_burned_area']
    df_temp.drop(columns=['20_burned_area','30_burned_area','40_burned_area','100_burned_area','110_burned_area','120_burned_area', '130_burned_area', '140_burned_area', '150_burned_area','160_burned_area','170_burned_area','180_burned_area'], inplace=True)
    replace_names={ '10_burned_area': 'no_data' + "_burned_area",
            '50_burned_area': 'tree_cover_broadleaved_evergreen' + "_burned_area",
            '60_burned_area': 'tree_cover_broadleaved_deciduous' + "_burned_area", 
            '70_burned_area': 'tree_cover_needleleaved_evergreen' + "_burned_area", 
            '80_burned_area': 'tree_cover_needleleaved_deciduous' + "_burned_area", 
            '90_burned_area': 'tree_cover_mixed_leave' + "_burned_area", }
    df_temp.rename(columns = replace_names, inplace = True)
    return df_temp

In [6]:
file = "data/MODIS_20010101.nc"
ds = nc.Dataset(file)
df_data = create_geoframe(ds)
df_data = add_country(df_data)
df_data

Loading formatted geocoded file...


,lon,lat,country
0,-179.875,89.875,RU
1,-179.625,89.875,RU
2,-179.375,89.875,RU
3,-179.125,89.875,RU
4,-178.875,89.875,RU
...,...,...,...
1036795,178.875,-89.875,AQ
1036796,179.125,-89.875,AQ
1036797,179.375,-89.875,AQ
1036798,179.625,-89.875,AQ


In [7]:
df_all_data = pd.concat([df_data,
            create_column(ds, 'burned_area'),
            create_column(ds, 'standard_error'),
            create_column(ds,'fraction_of_burnable_area'),
            create_column(ds,'fraction_of_observed_area'),
            create_column(ds, 'number_of_patches'),
            create_vegetation_class_breakdown(ds)], axis = 1, sort = False) 

In [8]:
date = file[-11:-7] + "-" + file[-7:-5] + "-" + file[-5:-3]
df_all_data['date'] = date

In [9]:
df_all_data

,lon,lat,country,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area,date
0,-179.875,89.875,RU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
1,-179.625,89.875,RU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
2,-179.375,89.875,RU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
3,-179.125,89.875,RU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
4,-178.875,89.875,RU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036795,178.875,-89.875,AQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
1036796,179.125,-89.875,AQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
1036797,179.375,-89.875,AQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01
1036798,179.625,-89.875,AQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001-01-01


In [ ]:
#Load to Database

In [13]:
with open('config/configLocalDS.json') as f:
    conf = json.load(f)

In [14]:
conn_str ='postgresql://%s:%s@localhost:5432/%s'%(conf["user"], conf["passw"],conf["database"])
engine = create_engine(conn_str)